In [ ]:
import os
import sys
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from tqdm import trange
%matplotlib inline

In [ ]:
from importlib import reload  
reload(latent3d_evaluation_metrics)
import latent3d_evaluation_metrics

In [ ]:
def pc_of_pc_to_grid(A):
    
    grid_resolution = 28 
    nn, grid_coordinates = latent3d_evaluation_metrics.get_nn_obj(grid_resolution = grid_resolution)
    
    G = np.zeros((A.shape[0], grid_resolution**3))
    
    for i in trange(A.shape[0]):
        G[i] = latent3d_evaluation_metrics.get_grid_counters([A[i]], grid_resolution = grid_resolution,\
                                                            nn = nn, grid_coordinates = grid_coordinates)
    return G

In [ ]:
A = np.load('/latent_3d/chairs.npz')['arr_0']
cloud_base = np.reshape(A, (6778, 2048 * 3))
cloud_base_orig = A
cloud_base_grid = pc_of_pc_to_grid(A)

In [ ]:
epochs = list(np.concatenate(([1, 5, 10, 50], np.arange(100, 1005, 100))))

In [ ]:
epochs

In [ ]:
clouds = []
clouds_orig = []
clouds_grid = []

for ep in epochs:
    A = np.load('/latent_3d/ep1000/epoch_%d.npz' % ep)['arr_0']
    clouds.append(np.reshape(A, (6778, 2048 * 3)))
    clouds_orig.append(A)
    clouds_grid.append(pc_of_pc_to_grid(A))

In [ ]:
cloud_base.shape

In [ ]:
for cloud in clouds:
    print(cloud.shape)

In [ ]:
cloud_base_grid.shape

In [ ]:
for cloud in clouds_grid:
    print(cloud.shape)

### Compute cross-barcodes 

In [ ]:
import mtd

In [ ]:
res1 = []
trials = 10

for i in trange(len(clouds)):
    np.random.seed(7)
    barcs = [mtd.calc_cross_barcodes(cloud_base_grid, clouds_grid[i], batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 1, pdist_device = 'gpu') for _ in range(trials)]
    res1.append(barcs)

In [ ]:
res2 = []
trials = 10

for i in trange(len(clouds)):
    np.random.seed(7)
    barcs = [mtd.calc_cross_barcodes(clouds_grid[i], cloud_base_grid, batch_size1 = 100, batch_size2 = 1000,\
                                          cuda = 1, pdist_device = 'gpu') for _ in range(trials)]
    res2.append(barcs)

### Absolute barcodes

In [ ]:
mtd.calc_cross_barcodes(clouds_grid[-1], np.zeros((0,0)), batch_size1 = 100, batch_size2 = 0,\
                                          cuda = 3, pdist_device = 'cpu')

In [ ]:
mtd.calc_cross_barcodes(cloud_base_grid, np.zeros((0,0)), batch_size1 = 100, batch_size2 = 0,\
                                          cuda = 3, pdist_device = 'cpu')

In [ ]:
def get_scores(res, args_dict, trials = 10):

    scores = []

    for i in range(len(res)): 
        asum = []
        
        for exp_id, elem in enumerate(res[i]):
            asum.append(mtd.get_score(elem, **args_dict))

        scores.append(sum(asum) / len(res[i]))

    return scores

In [ ]:
scores = get_scores(res1, {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
for ep, s in zip(epochs, scores):
    print(ep, s)

In [ ]:
scores = get_scores(res2, {'h_idx' : 1, 'kind' : 'sum_length'})

In [ ]:
for ep, s in zip(epochs, scores):
    print(ep, s)

In [ ]:
#pickle.dump(res1, open("res1_1000_100_ep1000_grid.pickle", 'wb'))
#pickle.dump(res2, open("res2_1000_100_ep1000_grid.pickle", 'wb'))

### PCA

In [ ]:
from sklearn.decomposition import PCA
%pylab inline

In [ ]:
# Create data
def plot2(data, groups = ("base", "cloud")):
    colors = ("red", "green")

    # Create plot
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for data, color, group in zip(data, colors, groups):
        x, y = data
        ax.scatter(x, y, alpha=0.5, c=color, edgecolors='none', s=5, label=group)

    #plt.title('Matplot scatter plot')
    plt.legend(loc=2)
    plt.show()

#### PCA from base+last GAN

In [ ]:
all_pca = []

for i in range(len(epochs)):
    pca = PCA(n_components=2)

    cb = np.concatenate((cloud_base_grid, clouds_grid[-1]))
    pca.fit(cb)

    cb = cloud_base_grid
    cloud_base_pca = pca.transform(cb)
    data = [(cloud_base_pca[:,0], cloud_base_pca[:,1])]

    cg = clouds_grid[i]

    cloud_pca = pca.transform(cg)
    data.append((cloud_pca[:,0], cloud_pca[:,1]))
    
    all_pca.append(data)

    plot2(data, groups = ("real", "generated, epoch %d" % epochs[i]))

In [ ]:
#pickle.dump(all_pca, open('3d_gan_pca.pickle', 'wb'))

### Classifier 2-sample test

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [ ]:
def c2st(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

    mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), learning_rate_init = 0.01)
    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)
    return accuracy_score(y_pred,y_test)

In [ ]:
for i in range(len(epochs)):
    X = np.zeros((6778*2, 21952))
    y = np.zeros((6778*2))

    X[0:6778,:] = cloud_base_grid
    X[6778:13556,:] = clouds_grid[i]

    y[0:6778] = 0
    y[6778:13556] = 1
    
    print(epochs[i], c2st(X, y))